## Identify Board Space

https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_corner.html#sphx-glr-auto-examples-features-detection-plot-corner-py

### Detect Edges

In [ ]:
%matplotlib inline

import itertools
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import ndimage as ndi
from skimage import io, feature, morphology
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage import transform as tf
from skimage.filters import threshold_otsu
from skimage.transform import resize, hough_line, hough_line_peaks, warp, ProjectiveTransform
from skimage import img_as_ubyte
from skimage.morphology import closing, square, opening


def load_image(image_path, resize_img=True, grayscale_img=False):
    image = io.imread(image_path)

    if resize_img:
        # resize image such that shortest edge is 480px
        if image.shape[1] < image.shape[0]:
            to_shape = (image.shape[0] / (image.shape[1] / 480), 480)
        else:
            to_shape = (480, image.shape[1] / (image.shape[0] / 480))
        image = resize(image, to_shape)

    if grayscale_img:
        image = rgb2gray(image)

    return image


def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False


def detect_board(image, plot=True):
    gray = rgb2gray(image)

    bw = gray > (threshold_otsu(gray))  # to black & white
    bw = opening(bw, square(9))  # remove isolated white spots
    filled = ndi.binary_fill_holes(bw) 

    # only keep shapes larger than 1/4 of the image area
    cleaned = morphology.remove_small_objects(
        filled, image.shape[0] * image.shape[1] / 4 
    )

    edge = canny(cleaned)  # get edges of large shape
    
    # get straight lines
    h, theta, d = hough_line(edge)
    _, angles, dists = hough_line_peaks(
        hspace=h, angles=theta, dists=d, num_peaks=4, 
        threshold=0.5
    )

    lines = []
    for angle, C in zip(angles, dists):
        # Ax + By = C
        A = np.cos(angle)
        B = np.sin(angle)
        lines.append((A, B, C))

    corners = []
    for L1, L2 in itertools.combinations(lines, 2):
        pt = intersection(L1, L2)
        if not pt:
            continue
        conditions = [
             pt[0] > -50, 
             pt[1] > -50, 
             pt[0] < image.shape[1] + 50,
             pt[1] < image.shape[0] + 50, 
        ]
        if all(conditions):
            corners.append(pt)
    
    if plot:
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
        ax[0][0].imshow(bw, cmap=plt.cm.gray)
        ax[0][1].imshow(cleaned, cmap=plt.cm.gray)
        ax[1][0].imshow(edge, cmap=plt.cm.gray)

        ax[1][1].imshow(image, cmap=plt.cm.gray)
        ax[1][1].plot([t[0] for t in corners], 
                      [t[1] for t in corners], '.r')

        for angle, dist in zip(angles, dists):
            y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
            y1 = (dist - image.shape[1] * np.cos(angle)) / np.sin(angle)
            ax[1][1].plot((0, image.shape[1]), (y0, y1), '-r')

        ax[1][1].set_xlim((0, image.shape[1]))
        ax[1][1].set_ylim((image.shape[0], 0))
        
        plt.tight_layout()
        plt.show()
    
    return corners


def average(ls):
    return sum(ls) / len(ls)


def sort_points(corners):
    centroid = (average([t[0] for t in corners]),
                average([t[1] for t in corners]))
    
    board_srt = [
        next(filter(lambda x: is_top_left(x, centroid), corners)),
        next(filter(lambda x: is_btm_left(x, centroid), corners)),
        next(filter(lambda x: is_btm_rght(x, centroid), corners)),
        next(filter(lambda x: is_top_rght(x, centroid), corners)),
    ]

    return board_srt


def is_top_left(pt, centroid):
    return pt[0] < centroid[0] and pt[1] < centroid[1]


def is_btm_left(pt, centroid):
    return pt[0] < centroid[0] and pt[1] > centroid[1]


def is_btm_rght(pt, centroid):
    return pt[0] > centroid[0] and pt[1] > centroid[1]


def is_top_rght(pt, centroid):
    return pt[0] > centroid[0] and pt[1] < centroid[1]


def ball_loc(image, x, y):
    """Given a square board image, returns ball array by position"""
    ball_size = image.shape[0] // 9
    
    ball = image[
        ball_size * y : ball_size * y + ball_size, 
        ball_size * x : ball_size * x + ball_size, 
    ]
    
    return ball


def perspective_fix(brd_image, brd_pts):
    n_px = 480
    src = np.array([[   0,    0], [   0, n_px], 
                    [n_px, n_px], [n_px,    0]])
    dst = np.array(sort_points(brd_pts))
    tform3 = ProjectiveTransform()
    tform3.estimate(src, dst)
    brd_image = warp(brd_image, tform3, output_shape=(n_px, n_px))

    crop = 40
    brd_image = brd_image[crop:-crop, crop:-crop]
    brd_image = resize(brd_image, (99, 99))

    return brd_image

def get_balls(image):
    """Expects square input image of board space"""
    # convert each ball shape to 1D array
    balls = []
    for y in range(9):
        for x in range(9):
            ball = ball_loc(image, x, y)
            balls.append(ball.flatten())
            
    return np.vstack(balls)

In [ ]:
balls = []
labels = np.array([])
warped_imgs = []

for config_set in Path('img').iterdir():
    img_paths = [x for x in config_set.iterdir() if str(x).lower().endswith('jpg')]

    # get labels from config.csv
    config = pd.read_csv(config_set / 'config.csv', header=None)
    config = config.fillna('empty')
    config = config.values.flatten()
    # multiply labels by number of images
    config = np.vstack([config] * len(img_paths)).flatten()
    labels = np.append(labels, config)

    for img_path in img_paths:
        image = load_image(str(img_path))
        brd_pts = detect_board(image, plot=False)
        
        if len(brd_pts) < 4:
            print('Error:', str(img_path), 'failed to locate corners')
            continue
        
        image = perspective_fix(image, brd_pts)
        warped_imgs.append(image)
        balls.append(get_balls(image))

In [ ]:
balls = np.vstack(balls)

In [ ]:
# show random ball to check that labels match
import random

idx = random.randint(0, len(labels))
print(labels[idx], idx)
plt.imshow(balls[idx].reshape(11, 11, 3));

In [ ]:
image = load_image(str(img_path))
brd_pts = detect_board(image)

In [ ]:
from IPython.display import clear_output

for img in warped_imgs:
    plt.imshow(img)
    plt.pause(0.05)
    input()
    clear_output()

plt.show()

## Preprocess

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
labs_to_ints= {
    'empty': 0,
    'dk blue': 1,
    'dk green': 2,
    'dk purple': 3,
    'lt blue': 4,
    'lt green': 5,
    'lt purple': 6,
    'orange': 7,
    'red': 8,
    'yellow': 9
}

ints_to_labs = {v: k for k, v in labs_to_ints.items()}

In [ ]:
label_ints = np.array([labs_to_ints[x] for x in labels])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    balls, label_ints, test_size=0.33, random_state=42)

## Build the DNN

In [ ]:
import tensorflow as tf

n_inputs = 11 * 11 * 3
n_hidden1 = 150
n_hidden2 = 150
n_outputs = 10

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [ ]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [ ]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y, logits=logits
    )
    loss = tf.reduce_mean(xentropy, name="loss")

In [ ]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
# train
n_epochs = 100
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        batch_i = 0
        for iteration in range(X_train.shape[0] // batch_size):
            X_batch = X_train[batch_i:batch_i+batch_size]
            y_batch = y_train[batch_i:batch_i+batch_size]
            batch_i += batch_size
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val, y: y_val})
        print(epoch, "Train accuracy", acc_train, "Val accuracy", acc_val)
        
    save_path = saver.save(sess, "my_model_final.ckpt")

In [ ]:
# predict
idx = random.randint(0, len(balls))
ball = balls[idx]
X_new_scaled = ball.reshape(1, ball.shape[0])

with tf.Session() as sess:
    saver.restore(sess, "my_model_final.ckpt")
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
    
print(idx, '\n'
    'predicted: ', label_set[y_pred[0]], '\n',
    '   actual: ', labels[idx], sep='')
plt.imshow(balls[idx].reshape(11, 11, 3));

In [ ]:
# predict from image
img_paths = [x for x in Path('img/config_3').iterdir()]
image = load_image(str(img_paths[3]))
plt.imshow(image);

In [ ]:
brd_pts = detect_board(image, plot=False)
image = perspective_fix(image, brd_pts)
plt.imshow(image);

In [ ]:
slots = get_balls(image)

with tf.Session() as sess:
    saver.restore(sess, "my_model_final.ckpt")
    Z = logits.eval(feed_dict={X: slots})
    y_pred = np.argmax(Z, axis=1)
    
np.array([ints_to_labs[x] for x in y_pred]).reshape(9, 9)

## Exploratory Code

In [ ]:
# display results
fig, axes = plt.subplots(nrows=len(edges)+1, ncols=1, figsize=(10, 10),
                         sharex=True, sharey=True)

axes[0].imshow(image, cmap=plt.cm.gray)
axes[0].set_title('original image', fontsize=15)

i = 0
for ax, edge in zip(axes[1:], edges):
    i += 1
    ax.imshow(edge, cmap=plt.cm.gray)
    ax.set_title(f'Canny filter, $\sigma={i}$', fontsize=15)

# fig.tight_layout()

plt.show();

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.filters import roberts, sobel, scharr, prewitt


edge_roberts = roberts(image)
edge_sobel = sobel(image)

fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True,
                       figsize=(8, 4))

ax[0].imshow(edge_roberts, cmap=plt.cm.gray)
ax[0].set_title('Roberts Edge Detection')

ax[1].imshow(edge_sobel, cmap=plt.cm.gray)
ax[1].set_title('Sobel Edge Detection')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu


image = get_image(grayscale_img=True)
thresh = threshold_otsu(image)
binary = image > thresh

fig, axes = plt.subplots(ncols=2, figsize=(8, 2.5))

axes[0].imshow(image, cmap=plt.cm.gray)
axes[0].set_title('Original')

axes[1].imshow(binary, cmap=plt.cm.gray)
axes[1].set_title('Thresholded')

plt.show()

In [ ]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage import data

import matplotlib.pyplot as plt
from matplotlib import cm



# Generating figure 1
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

ax[1].imshow(image, cmap=cm.gray)
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
    y1 = (dist - image.shape[1] * np.cos(angle)) / np.sin(angle)
    ax[1].plot((0, image.shape[1]), (y0, y1), '-r')
ax[1].set_xlim((0, image.shape[1]))
ax[1].set_ylim((image.shape[0], 0))
ax[1].set_axis_off()
ax[1].set_title('Detected lines')

plt.tight_layout()
plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

ax.imshow(image, cmap=cm.gray)
ax.plot([t[0] for t in corners], 
        [t[1] for t in corners],
        '.r')
plt.show();

In [ ]:
import math

def clockwiseangle_and_distance(point, refvec= [0, 1]):
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
    # If length is zero there is no angle
    if lenvector == 0:
        return -math.pi, 0
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle, lenvector
    # I return first the angle because that's the primary sorting criterium
    # but if two vectors have the same angle then the shorter distance should come first.
    return angle, lenvector

In [ ]:
origin = list(np.array(corners).mean(axis=0))
sorted(corners, key=clockwiseangle_and_distance)

In [ ]:
from skimage import transform as tf

# top left, bottom left, bottom right, top right
src = np.array([[0, 0], [0, 500], [500, 500], [500, 0]])
# TODO: automate the identification of these corners:
dst = np.array(corners)

tform3 = tf.ProjectiveTransform()
tform3.estimate(src, dst)
warped = tf.warp(image, tform3, output_shape=(500, 500))

fig, ax = plt.subplots(nrows=2, figsize=(10, 10))

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].plot(dst[:, 0], dst[:, 1], '.r')
ax[1].imshow(warped, cmap=plt.cm.gray)

plt.tight_layout()

plt.show()

## Using Contours

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import measure
from skimage.filters import threshold_otsu


def bbox_area(points):
    """X: 2D array"""
    if len(points.shape) != 2 or points.shape[1] != 2:
        raise ValueError(
            f"Points must be a (n,2), array but it has shape {points.shape}"
        )
    if points.shape[0] < 1:
        raise ValueError("Can't compute bounding box for empty coordinates")
    minx, miny = np.min(points, axis=0)
    maxx, maxy = np.max(points, axis=0)

    return (maxx - minx) * (maxy - miny)

image = get_image(grayscale_img=True)

thresh = threshold_otsu(image)
binary = image > thresh

# Find contours at a constant value of 0.1
contours = measure.find_contours(binary, 0.1)
# Get largest contour
contour = max(contours, key=bbox_area)

# Display the image and plot largest contour bounding box
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
ax.plot(contour[:, 1], contour[:, 0], linewidth=4)

plt.show()

In [ ]:
approximate_polygon?

In [ ]:
from skimage.measure import approximate_polygon, subdivide_polygon

appr_contour = approximate_polygon(contour, tolerance=20)

print(contour.shape, appr_contour.shape)

# Display the image and plot largest contour bounding box
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
ax.plot(appr_contour[:, 1], appr_contour[:, 0], linewidth=4)

plt.show()

### Identify Corners

In [ ]:
from pathlib import Path

from matplotlib import pyplot as plt

from skimage.feature import corner_harris, corner_subpix, corner_peaks

image = get_image()

In [ ]:
coords = corner_peaks(corner_harris(image), min_distance=5)
coords_subpix = corner_subpix(image, coords, window_size=13)

fig, ax = plt.subplots()
ax.imshow(image, interpolation='nearest', cmap=plt.cm.gray)
# ax.plot(coords[:, 1], coords[:, 0], '.b', markersize=3)
ax.plot(coords_subpix[:, 1], coords_subpix[:, 0], '+r', markersize=15)
plt.show()

## Rectify Board Space

https://scikit-image.org/docs/stable/auto_examples/applications/plot_geometric.html#sphx-glr-auto-examples-applications-plot-geometric-py

TODO: automate identification of board corners

In [ ]:
from pathlib import Path

import math
import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage import data
from skimage import transform as tf

img_paths = [x for x in Path('img').iterdir()]
board = io.imread(str(img_paths[0]))

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(board);

In [ ]:
# top left, bottom left, bottom right, top right
src = np.array([[0, 0], [0, 500], [500, 500], [500, 0]])
# TODO: automate the identification of these corners:
dst = np.array([
    [1250,   420], [930,  2420],  # x, y left edge
    [3590,  2490], [3260,  410],   # x, y right edge
])

tform3 = tf.ProjectiveTransform()
tform3.estimate(src, dst)
warped = tf.warp(board, tform3, output_shape=(500, 500))

fig, ax = plt.subplots(nrows=2, figsize=(10, 10))

ax[0].imshow(board, cmap=plt.cm.gray)
ax[0].plot(dst[:, 0], dst[:, 1], '.r')
ax[1].imshow(warped, cmap=plt.cm.gray)

plt.tight_layout()

plt.show()

### Resources

* https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10
* https://github.com/EdjeElectronics/TensorFlow-Object-Detection-on-the-Raspberry-Pi
* https://hackernoon.com/building-an-insanely-fast-image-classifier-on-android-with-mobilenets-in-tensorflow-dc3e0c4410d4
* http://matpalm.com/blog/counting_bees/
* https://www.amazon.com/gp/product/B01ER2SKFS
* https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet
* https://www.reddit.com/r/MachineLearning/comments/8dy6wi/p_live_object_detection_on_raspberry_pi_cpu_with/